In [1]:
#imports
import pandas as pd
import numpy as np
import statistics
from statistics import mode
from datetime import datetime,date,timedelta
import random
from scipy import stats


In [2]:
pd.set_option('display.max_rows', None)

# 2. Grouping customers together!
    

Now we will deal with clustering algorithms that will provide groups of clients which are similar among them.

To solve this task, you must accomplish the following stages:

# 2.1 Getting your data + feature engineering


###### 1.Access to the data found in this dataset (it is the same dataset from the previous question 1.1).

###### Organising the data

In [3]:
df=pd.read_csv(r"C:\Users\PC\Desktop\bank_transactions.csv") #reading the data
df = df.dropna() #Dropping NaN values
df=df[50000:] #I dont have enough memory to run all the data

In [4]:
#Later I will change all 1/1/1800 to random dates
#I changed all '1/1/1800' DOB's with "1/4/00" because there were no "1/4/00" in data and it prevents the later error that is 
#raised because of the date not fitting the format "%d/%m/%y". 
df.loc[df['CustomerDOB'] == '1/1/1800','CustomerDOB']=df['CustomerDOB'].apply(lambda x: "1/4/00")



In [5]:
#Changing TransactionDate column from string to datetime object
df["TransactionDate"]= df.apply(lambda row: datetime.strptime(row.TransactionDate, "%d/%m/%y"), axis=1)
#Changing CustomerDOB column from string to datetime object
df["CustomerDOB"]= df.apply(lambda row: datetime.strptime(row.CustomerDOB, "%d/%m/%y"), axis=1)



In [6]:
#In india minimum age for opening a bank account is 18 so If some year values are bigger than 2004 they are probably people who
#really are born after 1904 and this requires an offset
df.loc[df.CustomerDOB.dt.year > 2004, 'CustomerDOB'] = df.loc[df.CustomerDOB.dt.year > 2004, 'CustomerDOB'] - pd.DateOffset(years = 100)


In [7]:
LastDOB = df["CustomerDOB"].max()
FirstDOB= df["CustomerDOB"].min()

#This function Creates a random date between the first date of birth and the last date of birth of people in data
def RandomDate(FirstDOB,LastDOB):
    DOBPeriod=LastDOB-FirstDOB
    RandomDayCount=random.randint(0,DOBPeriod.days)
    randomdate=FirstDOB+timedelta(days=RandomDayCount)
    return randomdate




In [8]:
#By this code the values equal to 1/1/1800 which were changed into 1/4/00 are replaced by random dates between the first date
#of birth and the last date of birth of people in data. I think distributing faulty date randomly is logical because
#there will be no false importance for any date. 
df.loc[df['CustomerDOB'] == "1/4/00",'CustomerDOB']=df['CustomerDOB'].apply(lambda x: RandomDate(FirstDOB,LastDOB))


### 2.Sometimes, the features (variables, fields) are not given in a dataset but can be created from it. The previous step is known as feature engineering. For example, the original dataset has several transactions done by the same customer.Then, we suggest you to group data by the client (using CustomerId) and, based on it, create the following new features for each CustomerId:

a) Number of transactions

b) Number of transactions with a balance bigger than 100 USD

c) Average amount of the transactions

d) Average balance

e) Average difference between the balance and the transaction amount for each customer (this is mainly known in the banking world as utilisation).

f) Most common gender of the customer

h) Most frequent location of the customer

So, in the end, you should have for each CustomerID seven features.



In [9]:
check=df.sort_values(by=["CustAccountBalance"],ascending=False)[:1000]

In [10]:
#When we sort data by "CustomerDOB","CustGender", "CustLocation" or "CustAccountBalance", we can see there is multiple
#customerID's with same Date of Birth, Gender, Location and Balance so I find it logical to assume these similar data belongs to
#the same person.Instead of grouping data by CustomerID I group data considering columns with same value
#I decided to use this method after reading about CEM ŞİRİN's ideas on SLACK
check[:10]

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
656264,T656265,C3316830,1967-08-03,M,SHILLONG,115035495.1,2016-08-28,231112,28019.32
826721,T826722,C2216823,1967-08-03,M,SHILLONG,115035495.1,2016-09-04,141239,4699.00
879623,T879624,C4316889,1967-08-03,M,SHILLONG,115035495.1,2016-09-08,213612,4364.00
1002847,T1002848,C7716886,1967-08-03,M,SHILLONG,115035495.1,2016-09-14,153335,3604.00
1002072,T1002073,C5116828,1967-08-03,M,SHILLONG,115035495.1,2016-09-14,151418,5060.00
789085,T789086,C6416825,1967-08-03,M,SHILLONG,115035495.1,2016-09-05,100606,5864.00
876392,T876393,C8116818,1967-08-03,M,SHILLONG,115035495.1,2016-09-08,214355,3149.00
835641,T835642,C2916887,1967-08-03,M,SHILLONG,115035495.1,2016-09-04,93141,600.00
1001213,T1001214,C5916881,1967-08-03,M,SHILLONG,115035495.1,2016-09-14,160123,1599.00
1005069,T1005070,C3414471,1931-07-04,F,GURGAON,82244629.9,2016-09-14,165018,3403.00


In [11]:
#"CustAccountBalance" :lambda x:x,"TransactionDate": lambda x:x,"TransactionTime": lambda x:x ,"TransactionAmount (INR)": lambda x: x,"CustGender": lambda x:x,"CustLocation":lambda x:x

In [12]:
grouped=df.groupby(["CustomerDOB","CustAccountBalance","CustLocation","CustGender"])
grouped_df=grouped.agg({"TransactionID": lambda x:x,"CustomerID":lambda x:x,"CustomerDOB":lambda x:x,"CustAccountBalance" :lambda x:np.mean(x),"TransactionDate": lambda x:x,"TransactionAmount (INR)": lambda x:np.mean(x),"CustGender": lambda x:x,"CustLocation":lambda x:x})#{"Transaction#":lambda x: len(x)} )
#lambda x: list(pd.Series.unique(x))

In [13]:
grouped_df[:3]

,,,,TransactionID,CustomerID,CustomerDOB,CustAccountBalance,TransactionDate,TransactionAmount (INR),CustGender,CustLocation
CustomerDOB,CustAccountBalance,CustLocation,CustGender,,,,,,,,
1905-01-05,5139.73,CHENNAI,F,"[T374060, T379954, T908889, T909272, T995752, ...","[C1921885, C8221853, C8721829, C1121811, C8221...","[1905-01-05 00:00:00, 1905-01-05 00:00:00, 190...",5139.73,"[2016-08-13 00:00:00, 2016-08-13 00:00:00, 201...",1974.000,"[F, F, F, F, F, F, F, F]","[CHENNAI, CHENNAI, CHENNAI, CHENNAI, CHENNAI, ..."
1905-02-23,308733.08,BALOTRA,M,T450450,C8533253,1905-02-23 00:00:00,308733.08,2016-08-16 00:00:00,2184.000,M,BALOTRA
1905-05-07,15394.67,DELHI,M,"[T694698, T718330]","[C8782254, C1382220]","[1905-05-07 00:00:00, 1905-05-07 00:00:00]",15394.67,"[2016-08-31 00:00:00, 2016-09-03 00:00:00]",2581.305,"[M, M]","[DELHI, DELHI]"


In [14]:
#grouped_df=grouped.agg({"TransactionID": lambda x:x,"CustomerID":lambda x:x,"CustomerDOB":lambda x:x, "CustAccountBalance" :lambda x:x,"TransactionDate": lambda x:x,"TransactionTime": lambda x:x ,"TransactionAmount (INR)": lambda x: x,"CustGender": lambda x:x,"CustLocation":lambda x:x} )#{"Transaction#":lambda x: len(x)} )

In [15]:
#grouped_df=grouped

In [16]:
#grouped_df=grouped.agg({"TransactionID": lambda x:x,"CustomerID":lambda x:x,"CustomerDOB":lambda x:x, "CustAccountBalance" :lambda x:x,"TransactionDate": lambda x:x,"TransactionTime": lambda x:x ,"TransactionAmount (INR)": lambda x: x,"CustGender": lambda x:x,"CustLocation":lambda x:x} )#{"Transaction#":lambda x: len(x)} )

In [17]:
#a) Number of transactions
grouped_df["Transaction#"]=grouped_df["TransactionID"].map(lambda x:1 if type(x)==str else len(x) )

#b) Number of transactions with a balance bigger than 100 USD
#1 Indian Rupee equals 0.012 United States Dollar (7.12.2022)
grouped_df["BiggerThan100USD"]=grouped_df["TransactionAmount (INR)"].map(lambda x: 1 if type(x)==np.float64 and x>8229 else sum(map(lambda a : a>8229, x)) if type(x)==np.ndarray else 0 )

#c) Average amount of the transactions
grouped_df["AverageAmountTransaction"]=grouped_df["TransactionAmount (INR)"]

#d)Average amount of balance
grouped_df["AverageBalance"]=grouped_df["CustAccountBalance"]

#e) Average difference between the balance and the transaction amount for each customer (utilisation).
grouped_df["Utilization"]=grouped_df["AverageBalance"]-grouped_df["AverageAmountTransaction"]

#f)Most common gender of the customer
#I think "CustGender" column does not indicate genders of many customers with different genders but belongs to one person and
#the same gender is written as many as this persons Transaction count.
grouped_df["MostCommonGender"]=grouped_df["CustGender"].map(lambda x:x if type(x)==str else mode(x))

#h) Most frequent location of the customer
grouped_df["MostCommonLocation"]=grouped_df["CustLocation"].map(lambda x:x if type(x)==str else mode(x))


# 3.Consider at least 20 additional features that can be generated for each CustomerId. Describe each of them and why you believe they will be helpful. Moreover, add it to the previous dataset (the one with seven features). In the end, you should have for each CustomerID at least 27 features (7 recommended + 20 suggested by you).

Hints for feature engineering:

- Instead of only using the average, you could use other functions such as minimum, maximum, percentiles, etc.
- Think of adding filters to your features. For instance, in "Number of transactions with balance bigger than 100 USD”, 
  the filter was "bigger than 100 USD". Can you think about other filters, even including other variables?
- Think of including the information given by the fields transaction date and time in your calculations.
- Think about getting information from a customer’s birthday, even comparing it to other fields.
- You could also calculate percentages in your features. Think of a plausible set of percentages worth to be calculated.

# FEATURES
Age group which has the most balance in the group
    #it is known that wealth and age and correlated so this feature can be useful to catch patterns and it is useful for 
    banks to know which age group they are appealing
Age and Gender which has the most balance in the group

Location with the highest balance

###### 1-Maximum Transaction Amount
Knowing the highest transaction amount can tell us the understand the economic situation of the user


In [18]:
#grouped_df["MaximumTransactionAmount"]=grouped_df["TransactionAmount (INR)"].map(lambda x: x if type(x)==np.float64 else x.max() )
grouped_df["MaximumTransactionAmount"]=grouped.agg({"TransactionAmount (INR)": lambda x:np.max(x)})#{"Transaction#":lambda x: len(x)} )


###### 2-Minimum Transaction Amount
Knowing the lovest transaction amount can tell us the understand the economic situation of the user

In [19]:
#grouped_df["MinimumTransactionAmount"]=grouped_df["TransactionAmount (INR)"].map(lambda x: x if type(x)==np.float64 else x.min() )
grouped_df["MinimumTransactionAmount"]=grouped.agg({"TransactionAmount (INR)": lambda x:np.min(x)})#{"Transaction#":lambda x: len(x)} )


###### 3-First Transaction


In [20]:
grouped_df["FirstTransaction"]=grouped_df["TransactionDate"].map(lambda x: x if type(x)==pd.Timestamp else x.min() )

###### 4-Last Transaction


In [21]:
grouped_df["LastTransaction"]=grouped_df["TransactionDate"].apply(lambda x: x if type(x)==pd.Timestamp else x.max() )


###### 5-Recency
Knowing how recent a transaction is made can help us understand how relevant the transaction is 

In [22]:
FirstTransaction= grouped_df["FirstTransaction"].min()
LastTransaction = grouped_df["LastTransaction"].max()
TransactionPeriod=LastTransaction-FirstTransaction
print(f" first transaction= {FirstTransaction}\n last transaction= {LastTransaction}\n transaction period {TransactionPeriod.days}")             


 first transaction= 2016-08-01 00:00:00
 last transaction= 2016-10-21 00:00:00
 transaction period 81


Recency=(LastTransaction-grouped_df["TransactionDate"][1]).days/TransactionPeriod.days

In [23]:
#Recency score= 1-(last transaction-transaction date)/transaction period
grouped_df['RecencyScore'] = grouped_df.apply(lambda row: 1-((LastTransaction-row.TransactionDate).days/TransactionPeriod.days), axis=1)

######  6-Age

In [24]:
grouped_df['Age'] = grouped_df["CustomerDOB"].map(lambda x:2022-x.year )#agegroup(x) if type(x)==pd.Timestamp else agegroup(x[0])

###### 7-Age Group

In [25]:
def agegroup(year):
    if 1992<year.year<=2004:
        return "18-29"
    elif 1982<year.year<=1992:
        return "30-39"
    elif 1972<year.year<=1982:
        return "40-49"
    elif 1962<year.year<=1972:
        return "50-59"
    elif 1952<year.year<=1962:
        return "60-69"
    elif 1942<year.year<=1952:
        return "70-79"
    elif year.year<1942:
        return "bigger than 80"
    else:
        return "noagegroup"
    
grouped_df['AgeGroup'] = grouped_df["CustomerDOB"].map(lambda x:agegroup(x) if type(x)==pd.Timestamp else agegroup(x[0]))


###### 8-Wealth Ratio


In [26]:
AllBalance=sum(grouped_df["AverageBalance"])
grouped_df["WealthRatio"]=grouped_df["AverageBalance"]/AllBalance
#grouped_df.sort_values(by="WealthRatio",ascending=False)[:5]

###### 9-Transaction Volume

In [27]:
grouped_df['TransactionVolume'] = grouped.agg({"TransactionAmount (INR)": lambda x:np.sum(x)})#{"Transaction#":lambda x: len(x)} )


###### 10-Transactions Ratio

In [28]:
AllTransactions=sum(grouped_df["TransactionVolume"])
grouped_df["TransactionsRatio"]=grouped_df["TransactionVolume"]/AllTransactions

###### 11-Long Time User


In [29]:
grouped_df["LongTimeUser"]=grouped_df["TransactionDate"].map(lambda x:0 if type(x)==pd.Timestamp else (max(x)-min(x)).days)

###### 12-Long Time User Percentile

In [30]:
grouped_df.rename(columns = {'TransactionAmount (INR)':'TransactionAmountINR'}, inplace = True)

In [31]:
grouped_df["LongTimeUser%"]=grouped_df["LongTimeUser"].map(lambda row:stats.percentileofscore(grouped_df["TransactionVolume"]/100, row))

###### 13-Transaction Amount  Percentile

In [32]:
grouped_df["TransactionAmountINR%"]=grouped_df["TransactionAmountINR"].map(lambda row:stats.percentileofscore(grouped_df["TransactionAmountINR"], row))

###### 14-Wealth Ratio Percentile

In [33]:
grouped_df["WealthRatio%"]=grouped_df["WealthRatio"].map(lambda row:stats.percentileofscore(grouped_df["WealthRatio"], row))

###### 15-Transaction# Ratio Percentile

In [34]:
grouped_df["Transaction#%"]=grouped_df["Transaction#"].map(lambda row:stats.percentileofscore(grouped_df["Transaction#"], row))

###### 16-Balance Ratio Percentile

In [35]:
grouped_df["CustAccountBalance%"]=grouped_df["CustAccountBalance"].map(lambda row:stats.percentileofscore(grouped_df["CustAccountBalance"], row))

###### 17-Transaction Amount In Dollars


In [36]:
#1 Indian Rupee equals 0.012 United States Dollar (7.12.2022)
grouped_df["TransactionAmountUSD"]=grouped_df["TransactionAmountINR"].map(lambda x:0.012*x)


##### 18-Transaction Frequency by Months


In [37]:
def TransactionFreqOneMonth(TransactionDate):
    Months=[0,0,0,0,0,0,0,0,0,0,0,0]
    Months[TransactionDate.month-1]=+1
    return Months


def TransactionFrequency(TransactionDate):
    Months=[0,0,0,0,0,0,0,0,0,0,0,0]
    for i in TransactionDate:
        Months[(i.month)-1]=+1
    return Months

grouped_df["TransactionFrequencybyMonth"]=grouped_df.apply(lambda row:TransactionFreqOneMonth(row.TransactionDate) if type(row.TransactionDate)==pd.Timestamp else TransactionFrequency(row.TransactionDate),axis=1) 

###### 19-Most transaction count in a month 


In [38]:
months=["January","February","March","April","May","June","July","August", "September", "October", "November", "December"]

grouped_df["MostTransactionsInMonth"]=grouped_df["TransactionFrequencybyMonth"].map(lambda x:months[x.index(max(x))])

###### 20-Frequency Ratio of User


In [39]:
AllTransactions=sum(grouped_df["Transaction#"])
grouped_df["FrequencyRatio"]=grouped_df["Transaction#"]/AllTransactions

# 2.2 Choose your features (variables)!


#I choose to apply FAMD so I can use both categorical and Continious data

#I need to edit all transaction based features because it cannot used as caterogical data

Normalisation

In [ ]:
"""grouped_df["Transaction#"]=grouped_df["Transaction#"].map(lambda x:x/(max(grouped_df["Transaction#"])-min(grouped_df["Transaction#"]))
###### 1-Maximum Transaction Amount
grouped_df["MaximumTransactionAmount"]=grouped_df["MaximumTransactionAmount"].map(lambda x:x/(max(grouped_df["MaximumTransactionAmount"])-min(grouped_df["MaximumTransactionAmount"]))
###### 2-Minimum Transaction Amount
grouped_df["MinimumTransactionAmount"]=grouped_df["MinimumTransactionAmount"].map(lambda x:x/(max(grouped_df["MinimumTransactionAmount"])-min(grouped_df["MinimumTransactionAmount"]))
###### 3-First Transaction
grouped_df["FirstTransaction"]=grouped_df["FirstTransaction"].map(lambda x:x/(max(grouped_df["FirstTransaction"])-min(grouped_df["FirstTransaction"]))
###### 4-Last Transaction
grouped_df["LastTransaction"]=grouped_df["LastTransaction"].map(lambda x:x/(max(grouped_df["LastTransaction"])-min(grouped_df["LastTransaction"]))
###### 5-Recency
grouped_df['RecencyScore']=grouped_df["RecencyScore"].map(lambda x:x/(max(grouped_df["RecencyScore"])-min(grouped_df["RecencyScore"]))
grouped_df['Age']=grouped_df["Age"].map(lambda x:x/(max(grouped_df["Age"])-min(grouped_df["Age"]))
grouped_df["WealthRatio"]=grouped_df["WealthRatio"].map(lambda x:x/(max(grouped_df["WealthRatio"])-min(grouped_df["WealthRatio"]))
grouped_df['TransactionVolume']=grouped_df["TransactionVolume"].map(lambda x:x/(max(grouped_df["TransactionVolume"])-min(grouped_df["TransactionVolume"]))
grouped_df["TransactionsRatio"]=grouped_df["TransactionsRatio"].map(lambda x:x/(max(grouped_df["TransactionsRatio"])-min(grouped_df["TransactionsRatio"]))
grouped_df["LongTimeUser"]=grouped_df["LongTimeUser"].map(lambda x:x/(max(grouped_df["LongTimeUser"])-min(grouped_df["LongTimeUser"]))
grouped_df["LongTimeUser%"]=grouped_df["LongTimeUser%"].map(lambda x:x/(max(grouped_df["LongTimeUser%"])-min(grouped_df["LongTimeUser%"]))
grouped_df["TransactionAmountINR%"]=grouped_df["TransactionAmountINR%"].map(lambda x:x/(max(grouped_df["TransactionAmountINR%"])-min(grouped_df["TransactionAmountINR%"]))
grouped_df["WealthRatio%"]=grouped_df["WealthRatio%"].map(lambda x:x/(max(grouped_df["WealthRatio%"])-min(grouped_df["WealthRatio%"]))
grouped_df["Transaction#%"]=grouped_df["Transaction#%"].map(lambda x:x/(max(grouped_df["Transaction#%"])-min(grouped_df["Transaction#%"]))
grouped_df["CustAccountBalance%"]=grouped_df["CustAccountBalance%"].map(lambda x:x/(max(grouped_df["CustAccountBalance%"])-min(grouped_df["CustAccountBalance%"]))
grouped_df["TransactionAmountUSD"]=grouped_df["TransactionAmountUSD"].map(lambda x:x/(max(grouped_df["TransactionAmountUSD"])-min(grouped_df["TransactionAmountUSD"]))
grouped_df["TransactionFrequencybyMonth"]=grouped_df["TransactionFrequencybyMonth"].map(lambda x:x/(max(grouped_df["TransactionFrequencybyMonth"])-min(grouped_df["TransactionFrequencybyMonth"]))
grouped_df["MostTransactionsInMonth"]=grouped_df["MostTransactionsInMonth"].map(lambda x:x/(max(grouped_df["MostTransactionsInMonth"])-min(grouped_df["MostTransactionsInMonth"]))
grouped_df["FrequencyRatio"]=grouped_df["FrequencyRatio"].map(lambda x:x/(max(grouped_df["FrequencyRatio"])-min(grouped_df["FrequencyRatio"]))"""

In [52]:
grouped_df.pop("TransactionID")
grouped_df.pop("CustomerID")
grouped_df["CustomerDOB"]=grouped_df["CustomerDOB"].map(lambda x:x.strftime("%Y/%m/%d" ) if type(x)==pd.Timestamp else x[0].strftime("%Y/%m/%d" ))
grouped_df["CustAccountBalance"]=grouped_df["CustAccountBalance"].map(lambda x:x if type(x)==np.float64 else x[0])
grouped_df.pop("TransactionDate")
grouped_df.pop("TransactionAmountINR")
grouped_df["CustGender"]=grouped_df["CustGender"].map(lambda x:x if type(x)==str else x[0])
grouped_df["CustLocation"]=grouped_df["CustLocation"].map(lambda x:x if type(x)==str else x[0])
grouped_df["FirstTransaction"]=grouped_df["FirstTransaction"].map(lambda x:x.strftime("%Y/%m/%d" ) if type(x)==pd.Timestamp else x[0].strftime("%Y/%m/%d" ))
grouped_df["LastTransaction"]=grouped_df["LastTransaction"].map(lambda x:x.strftime("%Y/%m/%d" ) if type(x)==pd.Timestamp else x[0].strftime("%Y/%m/%d" ))
grouped_df["RecencyScore"]=grouped_df["RecencyScore"].map(lambda x:x if type(x)==float else x.mean())
grouped_df["BiggestTransactionDay"]=grouped_df["BiggestTransactionDay"].map(lambda x:x.strftime("%Y/%m/%d" ) if type(x)==pd.Timestamp else x[0].strftime("%Y/%m/%d" ))
grouped_df.pop("TransactionFrequencybyMonth")
grouped_df.drop(columns="TransactionVolumeINRbyMonth")
grouped_df.pop("TransactionVolumeINRbyMonth")
grouped_df["Age"]=grouped_df["Age"].map(lambda x:x if type(x)==int else x[0])



CustomerDOB  CustAccountBalance  CustLocation                         CustGender
1905-01-05   5.139730e+03        CHENNAI                              F             [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
1905-02-23   3.087331e+05        BALOTRA                              M             [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
1905-05-07   1.539467e+04        DELHI                                M             [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
1905-06-06   5.857210e+03        DELHI                                M             [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
1905-07-11   9.405380e+03        CHANDIGARH                           M             [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
1905-08-15   4.357332e+04        GURGAON                              F             [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
1905-09-01   1.079776e+04        MUMBAI                               F             [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
1905-09-25   3.783255e+05        RAJKOT                               F 

In [ ]:
from prince import FAMD
famd = FAMD(n_components =5, n_iter = 5, random_state = 50)
famd.fit(grouped_df)
famd.transform(grouped_df)
famd.plot_row_coordinates(grouped_df,figsize=(10, 10) )